### Часть 2
Используя скаченные данные и `Pandas` (или `PySpark`) посчитайте:
- явку (%) по всем ТИК, результат отсортировать по убыванию
- выберите произвольного кандидата и найдите тот избиратльный участок, на котором он получил наибольший результат  (%, учитывать участки на которых проголосовало больше 100 человек)
- найдите ТИК, где разница между УИК с наибольшей явкой и наименьшей максимальна
- посчитайте дисперсию по явке для каждого ТИК (учитывать УИК)
- для каждого кандидата посчитайте таблицу: результат (%, округленный до целого) - количество УИК, на которых кандидат получил данный результат
- визуализируйте отношение - явка на УИК $\rightarrow$ результат кандидата. 

In [58]:
import pandas as pd
df = pd.read_csv("hw4/elections_scrape.csv")
df.head(5)

,ТИК,УИК,"Число избирателей, внесенных в список избирателей на момент окончания голосования","Число избирательных бюллетеней, полученных участковой избирательной комиссией","Число избирательных бюллетеней, выданных избирателям в помещении для голосования в день голосования","Число избирательных бюллетеней, выданных избирателям, проголосовавшим вне помещения для голосования",Число погашенных избирательных бюллетеней,"Число избирательных бюллетеней, содержащихся в переносных ящиках для голосования","Число избирательных бюллетеней, содержащихся в стационарных ящиках для голосования",Число недействительных избирательных бюллетеней,Число действительных избирательных бюллетеней,Число утраченных избирательных бюллетеней,"Число избирательных бюллетеней, не учтенных при получении",Амосов Михаил Иванович,Беглов Александр Дмитриевич,Тихонова Надежда Геннадьевна
0,1,1,1803,1200,587,11,602,11,587,23,575,0,0,110,345,120
1,1,2,1466,1100,433,14,653,14,430,19,425,0,0,53,326,46
2,1,3,2092,1600,576,22,1002,22,576,15,583,0,0,155,332,96
3,1,4,1056,1000,318,4,678,4,318,13,309,0,0,67,171,71
4,1,5,1827,1400,495,8,897,8,493,16,485,0,0,137,266,82


In [136]:
#Переименуем колонки для более удобного обращения
df = df.rename(columns={"ТИК" : 'tik','УИК' : 'uik',"Число избирателей, внесенных в список избирателей на момент окончания голосования" : 'wholeNumOfElectors',"Число избирательных бюллетеней, полученных участковой избирательной комиссией" : 'wholeNumOfBallots',"Число избирательных бюллетеней, выданных избирателям в помещении для голосования в день голосования" : 'givenBallotsInside',"Число избирательных бюллетеней, выданных избирателям, проголосовавшим вне помещения для голосования" : 'givenBallotsOutside', 'Число погашенных избирательных бюллетеней' : 'ejectedBallots', "Число избирательных бюллетеней, содержащихся в переносных ящиках для голосования" : 'submittedBallotsOutside',"Число избирательных бюллетеней, содержащихся в стационарных ящиках для голосования" : 'submittedBallotsInside', 'Число недействительных избирательных бюллетеней' : 'spoiledBallots', 'Число действительных избирательных бюллетеней' : 'validBallots', 'Число утраченных избирательных бюллетеней' : 'lostBallots', "Число избирательных бюллетеней, не учтенных при получении" : 'untrackedBallots', 'Амосов Михаил Иванович' : 'amosov', 'Беглов Александр Дмитриевич' : 'beglov', 'Тихонова Надежда Геннадьевна' : 'tikhonova'})
df.head(5)

,tik,uik,wholeNumOfElectors,wholeNumOfBallots,givenBallotsInside,givenBallotsOutside,ejectedBallots,submittedBallotsOutside,submittedBallotsInside,spoiledBallots,validBallots,lostBallots,untrackedBallots,amosov,beglov,tikhonova,uik_turnout,tikhonova%,beglov%,amosov%
0,1,1,1803,1200,587,11,602,11,587,23,575,0,0,110,345,120,0.498333,21,60,19
1,1,2,1466,1100,433,14,653,14,430,19,425,0,0,53,326,46,0.406364,11,77,12
2,1,3,2092,1600,576,22,1002,22,576,15,583,0,0,155,332,96,0.373750,16,57,27
3,1,4,1056,1000,318,4,678,4,318,13,309,0,0,67,171,71,0.322000,23,55,22
4,1,5,1827,1400,495,8,897,8,493,16,485,0,0,137,266,82,0.359286,17,55,28


- Явка (%) по всем ТИК, результат отсортировать по убыванию

In [114]:
grouped = df.groupby(['tik']).sum()

turnout = (grouped['givenBallotsInside'] + grouped['givenBallotsOutside']) / (grouped['wholeNumOfElectors'])*100
turnout.sort_values(ascending=False)

tik
18    51.531228
15    40.013072
13    39.672610
19    37.812382
8     36.040981
20    33.780554
9     33.521975
22    32.486297
30    32.102011
2     31.787913
1     31.483272
29    30.770507
10    30.517413
4     30.431840
7     29.254120
21    29.138204
25    29.020921
17    28.440004
23    28.193842
16    28.191436
14    27.522452
28    27.224589
11    26.877083
5     26.753819
24    26.653968
3     26.620459
12    26.410227
26    26.264800
6     25.008379
27    24.792315
dtype: float64

- Участок, на котором Тихонова получила наибольший результат (%, учитывать участки на которых проголосовало больше 100 человек)

In [141]:
tikhonova_max = df[(df['tikhonova'] / (df['beglov'] + df['amosov'] + df['tikhonova']))]
tikhonova_max = tikhonova_max[(tikhonova_max['givenBallotsInside'] + tikhonova_max['givenBallotsOutside']) > 100]
df.loc[tikhonova_max.idxmax(), 'uik'], tikhonova_max.max()*100

KeyError: "None of [Float64Index([0.20869565217391303, 0.10823529411764705,  0.1646655231560892,\n               0.2297734627831715, 0.16907216494845362, 0.16079295154185022,\n               0.1865671641791045, 0.15924657534246575,  0.2095435684647303,\n              0.19624217118997914,\n              ...\n              0.18633540372670807,  0.1366906474820144, 0.11560693641618497,\n              0.11428571428571428, 0.06976744186046512,  0.0639269406392694,\n              0.11711711711711711, 0.09745762711864407,  0.0927536231884058,\n              0.09004739336492891],\n             dtype='float64', length=2021)] are in the [columns]"

- ТИК, где разница между УИК с наибольшей явкой и наименьшей максимальна

In [75]:
df['uik_turnout'] = (df['givenBallotsInside'] + df['givenBallotsOutside']) / df['wholeNumOfBallots']
tik_turnout_difference = grouped['uik_turnout'].max() - grouped['uik_turnout'].min()
tik_turnout_difference.idxmax(), tik_turnout_difference.max()

(23, 0.8233333333333333)

- Дисперсия по явке для каждого ТИК (учитывать УИК)

In [83]:
grouped['uik_turnout'].var()

tik
1     0.019097
2     0.020602
3     0.008367
4     0.006806
5     0.004582
6     0.003349
7     0.010603
8     0.017328
9     0.015448
10    0.011562
11    0.015875
12    0.004955
13    0.035405
14    0.006442
15    0.013576
16    0.007639
17    0.013702
18    0.027826
19    0.024000
20    0.015837
21    0.014933
22    0.010349
23    0.017824
24    0.008672
25    0.001570
26    0.014792
27    0.015475
28    0.018141
29    0.010332
30    0.029572
Name: uik_turnout, dtype: float64

- Таблица для каждого кандидата: результат (%, округленный до целого) - количество УИК, на которых кандидат получил данный результат

In [135]:
import numpy as np

df['tikhonova%'] = ((df['tikhonova'] / (df['beglov'] + df['amosov'] + df['tikhonova'])) * 100).round().astype('int')
df['beglov%'] = ((df['beglov'] / (df['beglov'] + df['amosov'] + df['tikhonova'])) * 100).round().astype('int')
df['amosov%'] = ((df['amosov'] / (df['beglov'] + df['amosov'] + df['tikhonova'])) * 100).round().astype('int')

grouped1 = df.groupby(['tikhonova%'])
tikhonova_table = grouped1['uik'].count()
grouped2 = df.groupby(['beglov%'])
beglov_table = grouped2['uik'].count()
grouped3 = df.groupby(['amosov%'])
amosov_table = grouped3['uik'].count()

ind = set(list(tikhonova_table.index))
[ind.add(i) for i in list(beglov_table.index)]
[ind.add(i) for i in list(amosov_table.index)]

table = []
for i in list(ind):
    try:
        var1 = tikhonova_table[i]
    except:
        var1 = np.nan
    try:
        var2 = beglov_table[i]
    except:
        var2 = np.nan
    try:
        var3 = amosov_table[i]
    except:
        var3 = np.nan
    table.append([var1, var2, var3])

table = pd.DataFrame(table, index=ind, columns=['tikhonova', 'beglov', 'amosov'])
table.index.name = 'percent'
table[:70:4]

,tikhonova,beglov,amosov
percent,,,
0,2.0,NaN,1.0
4,9.0,NaN,13.0
8,29.0,NaN,41.0
12,62.0,NaN,121.0
16,170.0,NaN,165.0
20,178.0,NaN,100.0
24,45.0,NaN,53.0
28,9.0,NaN,30.0
32,6.0,1.0,7.0


In [100]:
df[df['tikhonova%'] == 19]

,tik,uik,wholeNumOfElectors,wholeNumOfBallots,givenBallotsInside,givenBallotsOutside,ejectedBallots,submittedBallotsOutside,submittedBallotsInside,spoiledBallots,validBallots,lostBallots,untrackedBallots,amosov,beglov,tikhonova,uik_turnout,tikhonova%,beglov%,amosov%
6,1,7,2095,1600,532,17,1051,17,532,13,536,0,0,131,305,100,0.343125,19,57,24
20,1,21,1736,1300,522,17,761,17,522,21,518,0,0,138,283,97,0.414615,19,55,27
22,1,23,1344,1300,565,15,720,15,545,24,536,0,0,101,332,103,0.446154,19,62,19
45,1,46,1746,1300,507,6,787,6,507,17,496,0,0,87,314,95,0.394615,19,63,18
47,1,48,1204,1000,447,4,549,4,446,8,442,0,0,72,287,83,0.451000,19,65,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1987,30,2249,1858,1500,505,4,991,4,497,14,487,0,0,140,253,94,0.339333,19,52,29
1998,30,2260,1756,1500,517,7,976,7,517,21,503,0,0,126,280,97,0.349333,19,56,25
2000,30,2262,1433,1000,449,12,539,12,445,19,438,0,0,97,258,83,0.461000,19,59,22
2005,30,2267,1795,1500,534,7,959,7,524,18,513,0,0,97,318,98,0.360667,19,62,19
